In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 96% 302M/315M [00:05<00:00, 94.8MB/s]
100% 315M/315M [00:05<00:00, 56.1MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'kaggle.json',
 'rri-amplitudes.ipynb',
 'apnea-ecg-database.zip',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 462.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [56:48<00:00,  7.03s/it]

Testing...
x12: 100%|██████████| 527/527 [57:33<00:00,  6.55s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import  Dense,Flatten,MaxPooling2D,Conv2D,Permute,Reshape,LSTM,BatchNormalization,Bidirectional
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D,Conv2D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd
from keras.layers import GRU
from keras.layers import add, concatenate
from keras.layers import Reshape, Lambda


#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy
!pip install tensorflow
!pip install optimizers
!pip install keras

#from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from hyperactive.optimizers import RandomSearchOptimizer
from mealpy.swarm_based.GWO import OriginalGWO

from tensorflow.keras import optimizers
from keras.optimizers import Adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA

from tensorflow.python.keras import optimizers
#from keras.optimizers import RandomSearchOptimizer

import warnings
warnings.filterwarnings('ignore')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for optimizers: filename=Optimizers-0.1-py3-none-any.whl size=17005 sha256=ddf9000632a087de87a8c882d1a97905e22c32e1416e193626ecfd58c9c886bc
  Stored in directory: /root/.cache/pip/wheels/21/85/73/6114e547e45ab65253df3cba8144782286634fa9773a5a7828
Successfully built optimizers


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7],position[8]
    n1,n2,n3,n4,n5,n6,n7,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(n7),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs)

#def create_model(weight=1e-3):
    model= Sequential()
    model.add(Reshape((450,2,2),input_shape=(900,1,2)))
    model.add(Conv2D(n1, kernel_size=(11,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n2, kernel_size=(5,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1),strides=(2,1)))

    model.add(Permute((2,1,3)))
    model.add(Reshape((2,24*n5)))

    model.add(LSTM(n6, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(n7, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.01,76,246,364,364,246,44,14,30], # Lower bound of our parameters
    'ub':[0.001,128,512,512,712,512,74,32,200], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4, 1, 1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalGWO(epoch, pop_size)
  #t1 = time.time()

#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.005325594811688765 84 253 493 581 506 44 16 133


INFO:mealpy.swarm_based.GWO.OriginalGWO:Solving 9-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1.  1. ].


0.001 84 369 441 586 281 59 20 86
0.001 99 380 367 619 325 64 20 115
0.001 77 350 407 666 444 61 17 72
0.001 92 279 402 456 379 73 19 190
0.001 127 416 478 514 441 56 22 76
0.001 118 377 369 592 324 49 14 73
0.001 87 405 475 535 413 46 17 111
0.001 104 398 442 608 429 68 30 103
0.001 105 399 439 649 387 72 20 85
0.001 77 464 438 398 477 68 17 99
0.001 86 304 413 583 417 44 30 69
0.001 118 444 426 603 306 60 20 109
0.001 123 363 397 569 263 49 16 124
0.001 100 344 380 601 262 73 17 194
0.001 107 345 481 489 308 56 19 105
0.001 112 298 400 667 275 63 22 184
0.001 105 377 504 582 332 57 15 66
0.001 84 266 465 695 289 52 18 51
0.001 86 392 426 473 429 46 19 86
0.001 81 376 406 499 395 64 24 141
0.001 113 428 478 553 342 56 20 61
0.001 85 435 466 426 500 70 23 116
0.001 122 351 387 460 440 47 27 155
0.001 95 417 431 647 336 62 25 99
0.001 105 501 444 593 292 59 28 140
0.001 106 488 464 642 369 64 23 40
0.001 85 297 494 432 267 55 14 170
0.001 92 354 430 497 306 65 21 183
0.001 83 250 489 44

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 1, Current best: 1489.0004, Global best: 1489.0004, Runtime: 32.65179 seconds


0.001 79 337 512 534 246 72 32 30
0.001 128 246 364 364 262 74 32 36
0.001 76 246 423 459 246 54 14 44
0.001 128 427 364 481 335 44 30 30
0.001 128 246 364 364 288 74 14 48
0.001 76 246 364 464 262 46 32 30
0.001 116 388 364 410 490 44 14 30
0.001 118 246 364 364 251 48 26 30
0.001 111 280 364 599 246 70 22 53
0.001 124 286 364 364 246 74 20 108
0.001 76 352 364 429 483 46 14 30
0.001 99 273 421 452 246 44 19 30
0.001 109 246 512 429 246 74 32 60
0.001 112 246 364 441 460 59 32 47
0.001 76 246 451 364 246 74 16 32
0.001 110 396 512 364 258 74 22 66
0.001 76 335 453 364 246 56 18 45
0.001 128 246 376 364 246 57 22 89
0.001 81 352 364 397 246 63 32 65
0.001 124 315 364 712 265 44 29 30
0.001 76 362 364 364 246 48 32 42
0.001 108 304 512 460 307 46 32 66
0.001 76 246 497 364 246 44 32 77
0.001 76 289 449 364 377 74 30 31
0.001 90 433 512 364 246 74 14 30
0.001 76 304 375 642 313 69 20 30
0.001 128 363 450 495 317 52 25 30
0.001 76 246 364 491 246 44 19 56
0.001 119 246 365 405 274 44 19 3

...

0.001 76 247 365 364 246 44 14 30
0.001 76 246 367 365 246 44 14 30
0.001 76 247 364 364 246 44 14 30
0.001 76 246 364 365 246 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 365 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 247 364 364 247 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 247 365 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 366 364 246 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 246 366 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 247 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 247 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 366 364 246 44 14 30
0.001 76 246 366 364 246 44 14 30
0.001 76 246 3

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 99, Current best: 1384.0004, Global best: 1384.0004, Runtime: 31.89734 seconds


0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 364 364 246 44 14 30
0.001 76 246 3

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 100, Current best: 1384.0004, Global best: 1384.0004, Runtime: 30.18524 seconds


0.001 76 246 364 364 246 44 14 30
Solution: [1.00000000e-03 7.60000000e+01 2.46000000e+02 3.64000000e+02
 3.64000000e+02 2.46000000e+02 4.48517246e+01 1.40000000e+01
 3.00000000e+01], Fitness: (0.001, 76, 246, 364, 364, 246, 44, 14, 30)
